In [4]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('tweet_emotions.csv')

In [6]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [10]:
df.isnull().sum()

tweet_id     0
sentiment    0
content      0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [13]:
 print(df['sentiment'].value_counts())

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64


In [14]:
import re

def clean_text(text):
    text = text.lower()                       # lowercase
    text = re.sub(r'http\S+|www\S+', '', text) # remove URLs
    text = re.sub(r'@\w+', '', text)          # remove mentions
    text = re.sub(r'#\w+', '', text)          # remove hashtags
    text = re.sub(r'[^a-z\s]', '', text)      # remove punctuation/numbers
    text = text.strip()
    return text

df['clean_content'] = df['content'].apply(clean_text)


In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])  # converts classes to 0,1,2,...


In [16]:
from sklearn.model_selection import train_test_split

X = df['clean_content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


d:\DDownload\CICD\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.350375
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        31
           2       0.33      0.01      0.01       162
           3       0.00      0.00      0.00       163
           4       0.11      0.01      0.02       338
           5       0.34      0.36      0.35      1028
           6       0.49      0.14      0.22       268
           7       0.51      0.38      0.43       762
           8       0.34      0.56      0.42      1740
           9       0.35      0.02      0.04       352
          10       0.34      0.26      0.29      1046
          11       0.40      0.06      0.10       425
          12       0.33      0.49      0.39      1666

    accuracy                           0.35      8000
   macro avg       0.27      0.18      0.18      8000
weighted avg       0.34      0.35      0.32      8000



d:\DDownload\CICD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\DDownload\CICD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\DDownload\CICD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [25]:
import joblib

joblib.dump(model, "sentiment_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [26]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Assuming your dataset has 'sentiment' column
le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])

# Save the LabelEncoder
joblib.dump(le, "label_encoder.pkl")

# Check mapping
print(dict(zip(le.classes_, le.transform(le.classes_))))
# Example output: {'negative': 0, 'neutral': 1, 'positive': 2}


{'anger': np.int64(0), 'boredom': np.int64(1), 'empty': np.int64(2), 'enthusiasm': np.int64(3), 'fun': np.int64(4), 'happiness': np.int64(5), 'hate': np.int64(6), 'love': np.int64(7), 'neutral': np.int64(8), 'relief': np.int64(9), 'sadness': np.int64(10), 'surprise': np.int64(11), 'worry': np.int64(12)}
